### Data sources
- LEHD Origin-Destination Employment Statistics (LODES): The definition of variable codes, datasets, etc. can be found at the latest [LODES 7.3 Technical Documentation](https://lehd.ces.census.gov/data/lodes/LODES7/LODESTechDoc7.3.pdf). All LEHD Origin-Destination Employment Statistics (LODES) data are available, as described in the LODES documentation above. No changes have been made to the original CSV files. Data are available from 2002 to 2015. See the documentation above for caveats.
- Driving Times and Distances Dataset: Census tracts are 2010 vintage, and the columns are the origin tract, destination travel, travel time in minutes, and travel distance in miles. These data were calculated by the Data Science team at the Urban Institute. See [Github repo](https://github.com/UI-Research/spark-osrm).

In [1]:
import pyspark
from pyspark.sql.functions import *
from IPython.core.interactiveshell import InteractiveShell

warnings.filterwarnings(action='once')
InteractiveShell.ast_node_interactivity = "all"

In [2]:
spark = SparkSession.builder \
    .appName('pyspark-exploration') \
    .config('spark.driver.cores', '2') \
    .config('spark.executor.memory', '8gb') \
    .config('spark.executor.cores', '2') \
    .getOrCreate()     

In [3]:
def debug(df):
    """
    Function to pretty print the toDebugString
    """
    for rddstring in df.rdd.toDebugString().split('\n'):
        print rddstring.strip()

### Load and prepare data

Load in data and see what it looks like

In [28]:
drive = spark.read.parquet('s3://lsdm-emr-util/lsdm-data/travel-times/drive_times.parquet')
od = spark.read.parquet('s3://lsdm-emr-util/lsdm-data/lodes/od/od.parquet')

In [5]:
print((drive.count(), len(drive.columns)))
drive.take(2)
drive.dtypes

(122004331, 4)


[Row(from_tract=u'36103146402', to_tract=u'42091207003', miles=141.2, minutes=184.1),
 Row(from_tract=u'36103146402', to_tract=u'42091209000', miles=162.7, minutes=203.4)]

[('from_tract', 'string'),
 ('to_tract', 'string'),
 ('miles', 'double'),
 ('minutes', 'double')]

In [6]:
print((od.count(), len(od.columns)))
od.take(2)
od.dtypes

(1577789908, 14)


[Row(w_geocode=u'271630714002025', h_geocode=u'271630712082020', s000=1, sa01=0, sa02=1, sa03=0, se01=0, se02=1, se03=0, si01=1, si02=0, si03=0, createdate=u'20160219', year=2012),
 Row(w_geocode=u'271630714002025', h_geocode=u'271630712083004', s000=1, sa01=0, sa02=1, sa03=0, se01=0, se02=1, se03=0, si01=1, si02=0, si03=0, createdate=u'20160219', year=2012)]

[('w_geocode', 'string'),
 ('h_geocode', 'string'),
 ('s000', 'int'),
 ('sa01', 'int'),
 ('sa02', 'int'),
 ('sa03', 'int'),
 ('se01', 'int'),
 ('se02', 'int'),
 ('se03', 'int'),
 ('si01', 'int'),
 ('si02', 'int'),
 ('si03', 'int'),
 ('createdate', 'string'),
 ('year', 'int')]

Make census tract columns in origin-destination data

In [29]:
od = od.withColumn('h_tract', substring(od.h_geocode, 0, 11))\
        .withColumn('w_tract', substring(od.w_geocode, 0, 11))

Make "total" and "pct" columns

In [30]:
sa_cols = ['sa01', 'sa02', 'sa03']
se_cols = ['se01', 'se02', 'se03']
si_cols = ['si01', 'si02', 'si03']

In [31]:
od = od.withColumn('sa_total', od.sa01+od.sa02+od.sa03)\
    .withColumn('se_total', od.se01+od.se02+od.se03)\
    .withColumn('si_total', od.si01+od.si02+od.si03)

In [32]:
for cat_ls, cat_name in [(sa_cols, 'sa_total'), (se_cols, 'se_total'), (si_cols, 'si_total')]:
    for col in cat_ls:
        new = col + '_pct'
        od = od.withColumn(new, od[col]/od[cat_name])

In [33]:
od.columns

['w_geocode',
 'h_geocode',
 's000',
 'sa01',
 'sa02',
 'sa03',
 'se01',
 'se02',
 'se03',
 'si01',
 'si02',
 'si03',
 'createdate',
 'year',
 'h_tract',
 'w_tract',
 'sa_total',
 'se_total',
 'si_total',
 'sa01_pct',
 'sa02_pct',
 'sa03_pct',
 'se01_pct',
 'se02_pct',
 'se03_pct',
 'si01_pct',
 'si02_pct',
 'si03_pct']

Check out lineage and partitions of the two dataframes

In [34]:
debug(od)

(137) MapPartitionsRDD[53] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[52] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[51] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   FileScanRDD[50] at javaToPython at NativeMethodAccessorImpl.java:0 []


In [35]:
debug(drive)

(8) MapPartitionsRDD[57] at javaToPython at NativeMethodAccessorImpl.java:0 []
|  MapPartitionsRDD[56] at javaToPython at NativeMethodAccessorImpl.java:0 []
|  MapPartitionsRDD[55] at javaToPython at NativeMethodAccessorImpl.java:0 []
|  FileScanRDD[54] at javaToPython at NativeMethodAccessorImpl.java:0 []


In [36]:
od.rdd.getNumPartitions()
drive.rdd.getNumPartitions()

137

8

### Join origin-destination and driving dataframes

Repartition od before joining



In [37]:
od = od.repartition(8)

Left join join with driving, giving us travel times for each origin-destination pair.  
Assumption: travel time and distance for a census tract is the same for all its comprising block groups.

In [38]:
df = od.join(drive, [drive.from_tract == od.h_tract, drive.to_tract == od.w_tract])

Resulting dataframe is split across 200 partitions, as we can see from the getNumPartitions method

In [39]:
df.rdd.getNumPartitions()

200

In [40]:
debug(df)

(200) MapPartitionsRDD[73] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[72] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[71] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   ZippedPartitionsRDD2[70] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[64] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   ShuffledRowRDD[63] at javaToPython at NativeMethodAccessorImpl.java:0 []
+-(8) MapPartitionsRDD[62] at javaToPython at NativeMethodAccessorImpl.java:0 []
|  ShuffledRowRDD[61] at javaToPython at NativeMethodAccessorImpl.java:0 []
+-(137) MapPartitionsRDD[60] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[59] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   FileScanRDD[58] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[69] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   ShuffledRowRDD[68] at javaToPython at NativeMe

### Grouped aggregation - average travel time by year

In [ ]:
agg_df = df.groupBy("year").agg(avg("si01_pct"))
display(agg_df)
agg_df.collect()

DataFrame[year: int, avg(si01_pct): double]